# 07 — Daily report (end-to-end, using Universe Provider)

This notebook runs the full daily pipeline and exports a CSV.

New: tickers are loaded from the Universe Provider:
- `load_universe_from_package("mega")`
- optionally capped via `max_tickers` (like CLI `--top`)

Pipeline:
1) fetch OHLCV
2) build eligible universe
3) rank top N
4) compute entry signals
5) compute position sizing
6) generate final report

In [1]:
import pandas as pd
pd.set_option("display.width", 140)
pd.set_option("display.max_columns", 80)

## 1) Load universe tickers

In [2]:
from swing_screener.data.universe import load_universe_from_package, UniverseConfig

tickers = load_universe_from_package("mega", UniverseConfig(max_tickers=50))
tickers[:15], len(tickers)

(['AAPL',
  'MSFT',
  'NVDA',
  'AMZN',
  'META',
  'GOOGL',
  'TSLA',
  'AMD',
  'AVGO',
  'NFLX',
  'INTC',
  'SPY'],
 12)

## 2) Fetch market data

In [3]:
from swing_screener.data.market_data import fetch_ohlcv, MarketDataConfig

ohlcv = fetch_ohlcv(tickers, MarketDataConfig(start="2022-01-01"))
ohlcv.tail()

Open                                                                                                             \
                  AAPL        MSFT        NVDA        AMZN        META       GOOGL        TSLA         AMD        AVGO       NFLX   
Date                                                                                                                                
2026-01-07  263.200012  479.760010  188.570007  239.610001  655.640015  314.359985  435.899994  212.130005  341.700012  91.559998   
2026-01-08  257.019989  481.239990  189.110001  243.059998  645.880005  328.970001  427.890015  210.899994  342.779999  90.449997   
2026-01-09  259.079987  474.059998  185.080002  244.570007  645.440002  327.089996  435.950012  205.720001  337.920013  90.029999   
2026-01-12  259.160004  476.670013  183.220001  246.729996  652.530029  325.799988  441.230011  201.179993  340.950012  89.690002   
2026-01-13  259.410004  476.670013  184.960007  246.729996  641.700012  325.720001  441.225006  215.119995  352.720001  89.379997   

                                         High                                                                                      \
                 INTC         SPY        AAPL        MSFT        NVDA        AMZN        META       GOOGL        TSLA         AMD   
Date                                                                                                                                
2026-01-07  40.180000  692.190002  263.679993  489.700012  191.369995  245.289993  659.150024  326.149994  438.369995  212.130005   
2026-01-08  42.619999  688.820007  259.290009  482.660004  189.550003  246.410004  647.099976  330.320007  436.890015  210.940002   
2026-01-09  41.830002  690.630005  260.209991  479.820007  186.339996  247.860001  654.950012  330.829987  449.049988  207.300003   
2026-01-12  44.240002  690.679993  261.299988  480.989990  187.119995  248.940002  653.969971  334.040009  454.299988  209.880005   
2026-01-13  44.160000  695.489990  261.809998  475.760010  188.109894  247.660004  641.997498  340.480011  451.809998  223.110001   

                                                                 Low                                                              \
                  AVGO       NFLX       INTC         SPY        AAPL        MSFT        NVDA        AMZN        META       GOOGL   
Date                                                                                                                               
2026-01-07  349.700012  92.419998  44.570000  693.960022  259.809998  477.950012  186.559998  239.520004  644.809998  314.190002   
2026-01-08  343.679993  91.250000  43.340000  690.619995  255.699997  475.859985  183.710007  241.880005  635.719971  321.500000   
2026-01-09  347.390015  90.050003  45.730000  695.309998  256.220001  472.200012  183.669998  242.240005  642.849976  325.799988   
2026-01-12  353.899994  90.339996  45.450001  696.090027  256.799988  475.679993  183.020004  245.960007  641.229980  325.000000   
2026-01-13  359.489990  91.150002  48.240002  696.090027  258.390015  465.950012  183.410004  240.250000  624.099976  333.619995   

                                                                                       Close                                      \
                  TSLA         AMD        AVGO       NFLX       INTC         SPY        AAPL        MSFT        NVDA        AMZN   
Date                                                                                                                               
2026-01-07  431.290009  207.169998  335.880005  90.059998  40.119999  689.320007  260.329987  483.470001  189.110001  241.559998   
2026-01-08  424.369995  203.330002  330.500000  89.580002  40.680000  687.489990  259.040009  478.109985  185.039993  246.289993   
2026-01-09  430.390015  203.070007  333.500000  88.320000  41.570000  689.179993  259.369995  479.279999  184.860001  247.380005   
2026-01-12  438.000000  199.800003  340

## 3) Build daily report (end-to-end)

In [4]:
from swing_screener.reporting.report import build_daily_report, export_report_csv, today_actions, ReportConfig
from swing_screener.risk.position_sizing import RiskConfig

cfg = ReportConfig(
    risk=RiskConfig(account_size=500, risk_pct=0.01, k_atr=2.0, max_position_pct=0.60),
    only_active_signals=False,
)

report = build_daily_report(ohlcv, cfg)
report.head(20)

,rank,score,last,atr14,atr_pct,mom_6m,mom_12m,rs_6m,trend_ok,dist_sma50_pct,dist_sma200_pct,signal,breakout_level,ma20_level,entry,stop,shares,position_value,realized_risk
INTC,1,1.0,47.290001,2.205714,4.664229,1.063264,1.469452,0.941733,True,22.857338,67.793513,breakout,45.549999,38.968,47.29,42.88,1,47.29,4.41


## 4) Print a short action summary

In [ ]:
print(today_actions(report, max_rows=10))

## 5) Export CSV

In [ ]:
export_path = export_report_csv(report, "out/daily_report.csv")
export_path